In [1]:
import os
import re
import csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta

DATA_EYES = 'Data/P_Eyes/'
DATA_MIND = 'Data/P_Minds/'
DATA_BPN_PATH = 'Data/BPN_Data/'
DATA_LIKERT = 'Data/Likert.csv'

In [2]:
def Count_data(Area, data, Likert):

    data_Count = data['Att'].count()
    print('第 %s 區域的 專注度 資料筆數： %s' %(str(Area), str(data_Count)))
    
    data_Att_Avg = data['Att'].mean()
    print('第 %s 區域的 專注度 平均值： %s' %(str(Area), str(data_Att_Avg)))
    data_Att_Max = data['Att'].max()
    print('第 %s 區域的 專注度 最大值： %s' %(str(Area), str(data_Att_Max)))
    data_Att_Min = data['Att'].min()
    print('第 %s 區域的 專注度 最小值： %s' %(str(Area), str(data_Att_Min)))
    data_Att_Std = data['Att'].std()
    print('第 %s 區域的 專注度 標準差： %s' %(str(Area), str(data_Att_Std)))
    data_Att_Ske = data['Att'].skew()
    print('第 %s 區域的 專注度 偏移度： %s' %(str(Area), str(data_Att_Ske)))
    
    data_Att_upF, data_Att_downF, data_Att_updownF, area_F = count_F(data)
    print('第 %s 區域的 專注度上升次數 ： %s' %(str(Area), str(data_Att_upF)))
    print('第 %s 區域的 專注度下降次數 ： %s' %(str(Area), str(data_Att_downF)))
    print('第 %s 區域的 專注度平均變動 ： %s' %(str(Area), str(data_Att_updownF)))
    print('第 %s 區域的 專注度斜率面積 ： %s' %(str(Area), str(area_F)))
    
    data_Med_Avg = data['Med'].mean()
    print('第 %s 區域的 冥想度 平均值： %s' %(str(Area), str(data_Med_Avg)))
    
    ctime, freq = count_time(data)
    print('第 %s 區域的 注視時間： %s 秒' %(str(Area), str(ctime)))
    print('第 %s 區域的 注視次數： %s 秒' %(str(Area), str(freq)))
    print('第 %s 區域的 平均注視時間： %s 秒' %(str(Area), str(ctime//freq)))
    
    print('第 %s 區域的 喜歡程度： %s 分' %(str(Area), str(Likert)))
    
    if data_Att_Std:
        with open(DATA_BPN_PATH + '/All.csv', 'a') as f:
            result = (
                      '%.2f'%(data_Att_Std) + ',' +
                      str(data_Att_upF) + ',' +
                      str(data_Att_downF) + ',' +
                      '%.2f'%(data_Att_updownF) + ',' +
                      #str(area_F) + ','  +
                      str(ctime) + ',' +
                      str(freq) + ',' +
                      #'%.2F'%(ctime//freq) + ',' +
                      str(Likert) + '\n'
                      )
            f.write(result)
            
    if data_Att_Std:
        with open(DATA_BPN_PATH + '/Eye.csv', 'a') as f:
            result = (
                      str(ctime) + ',' +
                      str(freq) + ',' +
                      '%.2F'%(ctime//freq) + ',' +
                      str(Likert) + '\n'
                      )
            f.write(result)
    
    if data_Att_Std:
        with open(DATA_BPN_PATH + '/Mind.csv', 'a') as f:
            result = (
                      #'%.2f'%(data_Att_Std) + ',' +
                      str(data_Att_upF) + ',' +
                      str(data_Att_downF) + ',' +
                      '%.2f'%(data_Att_updownF) + ',' +
                      #str(area_F) + ',' +
                      str(Likert) + '\n'
                      )
            f.write(result)    
        

In [3]:
def count_time(data):

    start_time = 0 # 記錄每次注視開始的時間
    temp_time  = 0 # 借放上一次時間
    total_time = 0 # 計算總和的觀看時間
    count = 0      # 目前統計筆數
    Area = 0
    see_freq = 0   # 看該區域的次數

    # see: 
    # https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
    # iterrows 取得每一行
    for index, row in data.iterrows():
        # if not Ture -> False
        # if not False -> Ture
        # 如果int等於0時 ； String等於''時 ; Bool等於false -> False
        # if not Area=0 -> Ture
        if not Area:
            Area = row['Area'] # 懶得傳，用拿的
            
        # 先轉成時間格式方便計算，但毫秒已經損毀無法轉換故直接捨棄
        # strptime() 把「時間符串」解析為「時間元組」。
        time = datetime.strptime(row['Time'][:-2], '%H:%M:%S')

        # 如果已經有上一筆
        if temp_time:
            # 如果和上一筆間隔太久
            if (time - temp_time).seconds > 3:
                total_time += (temp_time - start_time).seconds # 總結上一次的時間長度
                start_time  = time # 把這次視為新的開始
                see_freq += 1

            # 如果這是最後一筆
            if count == (data['Time'].count() - 1):
                total_time += (time - start_time).seconds # 總結這次的時間長度

        # 如果是第一筆，紀錄為第一次的開始時間
        else:
            start_time = time
            see_freq += 1

        temp_time = time # 記下這次時間
        count += 1
        
        if not total_time:
            total_time = 1

    return total_time, see_freq

In [4]:
def cov(time_str):
    time_str = time_str[:-2]
    time = datetime.strptime(time_str, '%H:%M:%S')
    return time


def count_F(data):
    time = map(lambda x: cov(x), data['Time']) 
    time = list(time)

    att = list(data['Att'])

    start_i = 0
    target_t = time[start_i] +  timedelta(seconds = 5)
    att_list = []
    upup_times = 0
    downdown_times = 0
    a = 0

    for i in range(len(time)):
        if time[i] >= target_t or (i + 1) == len(time):

            # 取得最大最小值
            att_list = pd.Series(att[start_i: i])
            att_max       = att_list.max()
            att_max_index = att_list.idxmax()
            att_min       = att_list.min()
            att_min_index = att_list.idxmin()

            # 計算斜率
            x = att_max_index - att_min_index
            y = att_max - att_min
            f = y / x

            # 門檻
            if f >= 0.8 or f <= -0.8:

                # 計算次數
                is_upup = False
                if att_max_index > att_min_index:
                    is_upup = True
                    upup_times += 1
                else:
                    downdown_times += 1

                # 計算面積
                ta = (x * y) / 2
                if ta < 0:
                    ta = ta * -1
                a += ta

            # 準備下一個區間
            start_i = i
            target_t = time[start_i] +  timedelta(seconds = 5)

    tom = (upup_times - downdown_times)/(upup_times + downdown_times)
    
    return upup_times, downdown_times, tom, a

In [5]:
data_Likert = pd.read_csv(DATA_LIKERT)

if os.path.isfile(DATA_BPN_PATH + '/All.csv'):
    os.remove(DATA_BPN_PATH + '/All.csv')
    
data_Likert_Left_list = []
data_Likert_Right_list = []

for data_file in sorted(os.listdir(DATA_MIND)):
    if not data_file.startswith('.'):
        try:
            data_id = data_file[:-4] # 從檔名取出 ID ex.1234
            
            names_m = ['Time','Att','Med']
            data_m = pd.read_csv(DATA_MIND + data_id + '.csv', names = names_m)
            if data_m['Time'][0].startswith('Time:'):
                data_m['Time'] = data_m['Time'].map(lambda x: x[5:-6])
                data_m['Att'] = data_m['Att'].map(lambda x: int(x[4:]))
                data_m['Med'] = data_m['Med'].map(lambda x: int(x[4:]))
            else:
                data_m['Time'] = data_m['Time'].map(lambda x: x[0:-6])
            data_m.drop_duplicates(subset='Time', keep='last', inplace = True)

            names_e = ['Time','X','Y','Area']
            data_e = pd.read_csv(DATA_EYES + data_id + '.csv', names = names_e, encoding = 'UTF8', header = 0)
            if data_e['Time'][0].startswith('Time:'):
                data_e['Time'] = data_e['Time'].map(lambda x: x[5:-6])
                data_e['X'] = data_e['X'].map(lambda x: float(x[2:]))
                data_e['Y'] = data_e['Y'].map(lambda x: float(x[2:]))
                data_e['Area'] = data_e['Area'].map(lambda x: int(x[-1:]))
            else:
                data_e['Time'] = data_e['Time'].map(lambda x: x[0:-6])
            data_e.drop_duplicates(subset='Time', keep='last', inplace = True)
            
            data = pd.merge(data_m, data_e, on='Time', sort=True)

            data_group = data.groupby('Area')
            data_A1 = data_group.get_group(1) # 1 = 左邊
            data_A2 = data_group.get_group(2) # 2 = 右邊
            
            data_Likert_by_id = data_Likert[data_Likert['ID'].isin([data_id])] # 根據 ID 取出指定的列

            # 再從該 ID 的列裡面取出左右的五點量表的值
            data_Likert_Left  = data_Likert_by_id['Left'].values[0]
            data_Likert_Right = data_Likert_by_id['Right'].values[0]  
            
            
            if data_Likert_Left < 4:
                data_Likert_Left = 0
            else:
                data_Likert_Left = 1
                
            if data_Likert_Right < 4:
                data_Likert_Right = 0
            else:
                data_Likert_Right = 1
                 
                
            data_Likert_Left_list.append(data_Likert_Left)
            data_Likert_Right_list.append(data_Likert_Right)
            
            print('%s 計算資料如下：' %data_file)
            data_A1 = data_group.get_group(1) # 1 = 左邊
            Count_data(1, data_A1, data_Likert_Left)
            print("　")
            data_A2 = data_group.get_group(2) # 2 = 右邊
            Count_data(2, data_A2, data_Likert_Right)
            print("="*50)
        except:
            print('%s 發生錯誤，已跳過' %data_file)
            print("="*50)
            continue

1275.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 353
第 1 區域的 專注度 平均值： 57.20679886685552
第 1 區域的 專注度 最大值： 91
第 1 區域的 專注度 最小值： 11
第 1 區域的 專注度 標準差： 18.27736924098658
第 1 區域的 專注度 偏移度： -0.35312041479752776
第 1 區域的 專注度上升次數 ： 4
第 1 區域的 專注度下降次數 ： 6
第 1 區域的 專注度平均變動 ： -0.2
第 1 區域的 專注度斜率面積 ： 3514.5
第 1 區域的 冥想度 平均值： 64.40793201133144
第 1 區域的 注視時間： 58 秒
第 1 區域的 注視次數： 1 秒
第 1 區域的 平均注視時間： 58 秒
第 1 區域的 喜歡程度： 1 分
　
第 2 區域的 專注度 資料筆數： 209
第 2 區域的 專注度 平均值： 55.3732057416268
第 2 區域的 專注度 最大值： 91
第 2 區域的 專注度 最小值： 7
第 2 區域的 專注度 標準差： 24.012906180458568
第 2 區域的 專注度 偏移度： -0.23635990047172253
第 2 區域的 專注度上升次數 ： 4
第 2 區域的 專注度下降次數 ： 2
第 2 區域的 專注度平均變動 ： 0.3333333333333333
第 2 區域的 專注度斜率面積 ： 2013.0
第 2 區域的 冥想度 平均值： 47.86124401913876
第 2 區域的 注視時間： 29 秒
第 2 區域的 注視次數： 2 秒
第 2 區域的 平均注視時間： 14 秒
第 2 區域的 喜歡程度： 0 分
1459.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 407
第 1 區域的 專注度 平均值： 72.007371007371
第 1 區域的 專注度 最大值： 100
第 1 區域的 專注度 最小值： 47
第 1 區域的 專注度 標準差： 15.179790970524348
第 1 區域的 專注度 偏移度： 0.1402485202489891
第 1 區域的 專注度上升次數 ： 2
第 1 區域的 專注度下降次數 ： 3
第 1 區

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in long_scalars



第 1 區域的 注視次數： 3 秒
第 1 區域的 平均注視時間： 14 秒
第 1 區域的 喜歡程度： 1 分
　
第 2 區域的 專注度 資料筆數： 126
第 2 區域的 專注度 平均值： 54.20634920634921
第 2 區域的 專注度 最大值： 96
第 2 區域的 專注度 最小值： 38
第 2 區域的 專注度 標準差： 15.095068047712779
第 2 區域的 專注度 偏移度： 1.5047046817769325
第 2 區域的 專注度上升次數 ： 1
第 2 區域的 專注度下降次數 ： 2
第 2 區域的 專注度平均變動 ： -0.3333333333333333
第 2 區域的 專注度斜率面積 ： 476.5
第 2 區域的 冥想度 平均值： 52.53968253968254
第 2 區域的 注視時間： 18 秒
第 2 區域的 注視次數： 2 秒
第 2 區域的 平均注視時間： 9 秒
第 2 區域的 喜歡程度： 0 分
1598.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 716
第 1 區域的 專注度 平均值： 47.685754189944134
第 1 區域的 專注度 最大值： 74
第 1 區域的 專注度 最小值： 27
第 1 區域的 專注度 標準差： 9.278917730833527
第 1 區域的 專注度 偏移度： 0.3461980489818813
第 1 區域的 專注度上升次數 ： 4
第 1 區域的 專注度下降次數 ： 7
第 1 區域的 專注度平均變動 ： -0.2727272727272727
第 1 區域的 專注度斜率面積 ： 1626.5
第 1 區域的 冥想度 平均值： 56.625698324022345
第 1 區域的 注視時間： 101 秒
第 1 區域的 注視次數： 3 秒
第 1 區域的 平均注視時間： 33 秒
第 1 區域的 喜歡程度： 1 分
　
第 2 區域的 專注度 資料筆數： 153
第 2 區域的 專注度 平均值： 44.42483660130719
第 2 區域的 專注度 最大值： 61
第 2 區域的 專注度 最小值： 30
第 2 區域的 專注度 標準差： 8.740929742835018
第 2 區域的 專注度 偏移度： -0.021

4529.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 422
第 1 區域的 專注度 平均值： 46.06635071090047
第 1 區域的 專注度 最大值： 84
第 1 區域的 專注度 最小值： 13
第 1 區域的 專注度 標準差： 18.0871565159253
第 1 區域的 專注度 偏移度： 0.47637993385352145
第 1 區域的 專注度上升次數 ： 6
第 1 區域的 專注度下降次數 ： 6
第 1 區域的 專注度平均變動 ： 0.0
第 1 區域的 專注度斜率面積 ： 3284.5
第 1 區域的 冥想度 平均值： 72.26777251184834
第 1 區域的 注視時間： 56 秒
第 1 區域的 注視次數： 5 秒
第 1 區域的 平均注視時間： 11 秒
第 1 區域的 喜歡程度： 1 分
　
第 2 區域的 專注度 資料筆數： 248
第 2 區域的 專注度 平均值： 35.91129032258065
第 2 區域的 專注度 最大值： 81
第 2 區域的 專注度 最小值： 7
第 2 區域的 專注度 標準差： 15.475990074100952
第 2 區域的 專注度 偏移度： 0.19238738502340366
第 2 區域的 專注度上升次數 ： 4
第 2 區域的 專注度下降次數 ： 6
第 2 區域的 專注度平均變動 ： -0.2
第 2 區域的 專注度斜率面積 ： 1876.0
第 2 區域的 冥想度 平均值： 77.43145161290323
第 2 區域的 注視時間： 36 秒
第 2 區域的 注視次數： 5 秒
第 2 區域的 平均注視時間： 7 秒
第 2 區域的 喜歡程度： 1 分
4543.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 169
第 1 區域的 專注度 平均值： 51.28402366863905
第 1 區域的 專注度 最大值： 87
第 1 區域的 專注度 最小值： 1
第 1 區域的 專注度 標準差： 23.42317315942866
第 1 區域的 專注度 偏移度： -0.5951072195302187
第 1 區域的 專注度上升次數 ： 2
第 1 區域的 專注度下降次數 ： 3
第 1 區域的 專注度平均變動 ： -0.2


第 2 區域的 注視時間： 17 秒
第 2 區域的 注視次數： 2 秒
第 2 區域的 平均注視時間： 8 秒
第 2 區域的 喜歡程度： 0 分
65.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 366
第 1 區域的 專注度 平均值： 68.02459016393442
第 1 區域的 專注度 最大值： 100
第 1 區域的 專注度 最小值： 1
第 1 區域的 專注度 標準差： 29.37181477635033
第 1 區域的 專注度 偏移度： -0.9977328407723519
第 1 區域的 專注度上升次數 ： 2
第 1 區域的 專注度下降次數 ： 1
第 1 區域的 專注度平均變動 ： 0.3333333333333333
第 1 區域的 專注度斜率面積 ： 2026.0
第 1 區域的 冥想度 平均值： 32.89344262295082
第 1 區域的 注視時間： 38 秒
第 1 區域的 注視次數： 2 秒
第 1 區域的 平均注視時間： 19 秒
第 1 區域的 喜歡程度： 1 分
　
第 2 區域的 專注度 資料筆數： 476
第 2 區域的 專注度 平均值： 51.54621848739496
第 2 區域的 專注度 最大值： 94
第 2 區域的 專注度 最小值： 1
第 2 區域的 專注度 標準差： 22.987867884616524
第 2 區域的 專注度 偏移度： 0.22690205252362147
第 2 區域的 專注度上升次數 ： 3
第 2 區域的 專注度下降次數 ： 3
第 2 區域的 專注度平均變動 ： 0.0
第 2 區域的 專注度斜率面積 ： 4962.5
第 2 區域的 冥想度 平均值： 23.369747899159663
第 2 區域的 注視時間： 49 秒
第 2 區域的 注視次數： 1 秒
第 2 區域的 平均注視時間： 49 秒
第 2 區域的 喜歡程度： 1 分
659.csv 計算資料如下：
第 1 區域的 專注度 資料筆數： 666
第 1 區域的 專注度 平均值： 61.669669669669666
第 1 區域的 專注度 最大值： 100
第 1 區域的 專注度 最小值： 21
第 1 區域的 專注度 標準差： 19.090734174444133
第 1 區域的

In [6]:
pdd = pd.DataFrame()
pdd['Left'] = data_Likert_Left_list
pdd['Right'] = data_Likert_Right_list

print(pdd['Left'].value_counts().sort_index())
print('avg = %.2f' %pdd['Left'].mean())
print('='*50)
print(pdd['Right'].value_counts().sort_index())
print('avg = %.2f' %pdd['Right'].mean())

0     4
1    31
Name: Left, dtype: int64
avg = 0.89
0    25
1    10
Name: Right, dtype: int64
avg = 0.29
